<a href="https://colab.research.google.com/github/hood-boi/world-news-chatbot/blob/master/Validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
import torchtext
import random
import requests
import json
import time

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
text_field = torchtext.data.Field(sequential=True,      # text sequence
                                  tokenize=lambda x: x, # because are building a character-RNN
                                  include_lengths=True, # to track the length of sequences, for batching
                                  batch_first=True,
                                  use_vocab=True,
                                  init_token="<BOS>",
                                  eos_token="<EOS>"
                                 )       # to turn each character into an integer index
label_field = torchtext.data.Field(sequential=True,    # text sequence
                                   use_vocab=True,     # don't need to track vocabulary
                                   is_target=True,      
                                   batch_first=True,
                                   tokenize=lambda x: x,
                                   preprocessing=lambda x: x,
                                   init_token="<BOS>",
                                   eos_token="<EOS>"
                                  ) 

fields = [('reply', label_field), ('context', text_field)]
dataset1 = torchtext.data.TabularDataset("/content/gdrive/My Drive/Chatbot/rtodayilearned.txt", "tsv", fields); # name of the file
dataset2 = torchtext.data.TabularDataset("/content/gdrive/My Drive/Chatbot/rAskReddit.txt", "tsv", fields); # name of the file
dataset3 = torchtext.data.TabularDataset("/content/gdrive/My Drive/Chatbot/rnews.txt", "tsv", fields); # name of the file
dataset4 = torchtext.data.TabularDataset("/content/gdrive/My Drive/Chatbot/rworldnews.txt", "tsv", fields); # name of the file
dataset5 = torchtext.data.TabularDataset("/content/gdrive/My Drive/Chatbot/routoftheloop.txt", "tsv", fields); # name of the file
dataset6 = torchtext.data.TabularDataset("/content/gdrive/My Drive/Chatbot/rnottheonion.txt", "tsv", fields); # name of the file
dataset7 = torchtext.data.TabularDataset("/content/gdrive/My Drive/Chatbot/rCasualConversation.txt", "tsv", fields); # name of the file
dataset8 = torchtext.data.TabularDataset("/content/gdrive/My Drive/Chatbot/text.txt", "tsv", fields);
dataset = torch.utils.data.ConcatDataset([dataset1, dataset2, dataset3, dataset4, dataset5, dataset6, dataset7, dataset8])

train = torchtext.data.Dataset(dataset, fields)
text_field.build_vocab(train)
label_field.build_vocab(train)
print(text_field.vocab.itos)
print(label_field.vocab.itos)
input_vocab_size = len(text_field.vocab.itos)
reply_vocab_size = len(label_field.vocab.itos)
print("Input Vocab Size: ", input_vocab_size)
print("Reply Vocab size: ", reply_vocab_size)

['<unk>', '<pad>', '<BOS>', '<EOS>', ' ', 'e', 't', 'a', 'o', 'i', 'n', 's', 'r', 'h', 'l', 'd', 'u', 'c', 'm', 'g', 'y', 'p', 'f', 'w', 'b', '.', 'k', 'v', ',', 'I', 'T', 'S', 'A', 'C', 'W', '-', '/', '0', '?', 'M', 'x', 'j', 'P', 'H', 'B', 'N', 'D', 'E', '1', ':', 'R', 'F', 'O', '2', 'z', 'G', 'L', 'U', '*', '!', 'Y', 'J', 'K', '5', ')', '(', 'q', '3', '4', ';', '9', '8', '6', '7', 'V', '_', '$', ']', '[', '%', '=', '^', '~', 'Z', '&', 'Q', 'X', '#', '+', '\\', '|', '@', '`', '{', '}']
['<unk>', '<pad>', '<BOS>', '<EOS>', ' ', 'e', 't', 'o', 'a', 'i', 'n', 's', 'r', 'h', 'l', 'd', 'u', 'c', 'm', 'y', 'g', 'w', 'p', 'f', '.', 'b', 'k', 'v', 'I', ',', 'T', 'A', 'S', '/', '?', 'j', 'W', '-', 'x', 'H', 'C', '0', 'M', 'N', '!', 'B', 'E', 'O', 'D', 'P', 'R', '1', 'Y', ':', '*', 'F', 'z', 'L', 'G', '2', 'U', ')', '(', 'q', ';', 'J', '5', '3', 'K', '4', '9', '_', '8', '6', '7', 'V', ']', '[', '^', '$', '%', '=', '~', '&', 'Q', 'Z', 'X', '#', '\\', '+', '@', '|', '`', '}', '{']
Input Vocab Si

# >>> SEQ2SEQ

In [0]:
class ChatBot(nn.Module):
    def __init__(self, 
                 context_vocab_size,  
                 reply_vocab_size, 
                 encoder_hidden_size = 100,
                 generator_hidden_size = 100, 
                 encoder_layers = 1, 
                 generator_layers = 1):
        
        super(ChatBot, self).__init__()
        
        self.encoder_layers = encoder_layers;
        self.generator_layers =generator_layers;
        self.encoder_hidden_size = encoder_hidden_size;
        self.generator_hidden_size = generator_hidden_size;
        
        # >>> Encoder
        self.context_ident = torch.eye(context_vocab_size)
        self.encode_rnn = nn.LSTM(context_vocab_size, encoder_hidden_size, encoder_layers, batch_first=True)
        
        # >>> Generator
        self.reply_ident = torch.eye(reply_vocab_size)
        self.decode_rnn = nn.LSTM(reply_vocab_size, generator_hidden_size, generator_layers, batch_first=True)
        self.fcnn = nn.Linear(generator_hidden_size, reply_vocab_size)
        
    def forward(self, context, response, hidden=None):
        
        # >>> Encoder
        context_tensor = self.context_ident[context] # Type: batch.context[0] | Size: (batch size, sequence size)
        encode_out, encode_last_hidden = self.encode_rnn(context_tensor)
        
        # >>> Generator
        reply_tensor = self.reply_ident[response] #Type: batch.reply
        if(hidden == None):
            gen_out, gen_last_hidden = self.decode_rnn(reply_tensor, encode_last_hidden)
        else:
            gen_out, gen_last_hidden = self.decode_rnn(reply_tensor, hidden)
        out = self.fcnn(gen_out)
        return out, gen_last_hidden

In [0]:
def sample_sequence(model, context, text_field_itos, label_field_itos, text_field_stoi, label_field_stoi, max_len=100, temperature=0.1):
    generated_sequence = ""
    
    data_inp = ["<BOS>"] + list(context) + ["<EOS>"];
    data_inp_indices = [text_field_stoi[ch] for ch in data_inp];
    data_inp_tensor = torch.Tensor(data_inp_indices).long().unsqueeze(0);
    
    inp = torch.Tensor([label_field_stoi["<BOS>"]]).long()
    hidden = None;
    for p in range(max_len):
        #print(inp)
        output, hidden = model(data_inp_tensor, inp.unsqueeze(0), hidden)
        #print(output)
        #print(output.shape)
        
        #output = F.softmax(output, dim=2)
        #print(torch.argmax(output, dim=2))
        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        #print(output_dist)
        top_i = int(torch.multinomial(output_dist, 1)[0])
        # Add predicted character to string and use as next input
        predicted_char = label_field_itos[top_i]
        
        if predicted_char == "<EOS>":
            break
        generated_sequence += predicted_char       
        inp = torch.Tensor([top_i]).long()
    return generated_sequence

In [0]:
test_model = ChatBot(input_vocab_size, reply_vocab_size, encoder_layers = 2, generator_layers = 2)
test_model.load_state_dict(torch.load('/content/gdrive/My Drive/Chatbot/MM_WEIGHTS_12'))
test_model.eval()

ChatBot(
  (encode_rnn): LSTM(95, 100, num_layers=2, batch_first=True)
  (decode_rnn): LSTM(95, 100, num_layers=2, batch_first=True)
  (fcnn): Linear(in_features=100, out_features=95, bias=True)
)

In [0]:
def seq2seq(input):
    response = sample_sequence(test_model, input, text_field.vocab.itos, label_field.vocab.itos, text_field.vocab.stoi, label_field.vocab.stoi, temperature = 0.5, max_len=500);
    return response

In [0]:
seq2seq("Ajith Pai. Ajith Pai. Ajith Pai. Ajith Pai. Ajith Pai. Ajith Pai. Ajith Pai. Ajith Pai. Ajith Pai.")

'Yeah, I got a real state and the proper confused that was to say, and they are not saying it is such a surprise in the end of the control the thread in the last pain. I think they were a cops are supposedly assume it has a full of pain for profit. Its a different day where it was from the most part of the people to open a fun that are allowed to pay personal college people on the case then because the manufacturers was a lot of the shit are saying other than the same stuff and stuff a lot of peo'

# >>> BASELINE

In [0]:
class EncoderAE(nn.Module):
    def __init__(self,
                vocab_size,
                hidden_size = 100,
                hidden_layers = 1):
        
        super(EncoderAE, self).__init__()
        
        self.vocab_size = vocab_size;
        self.layers = hidden_layers;
        self.hidden_size = hidden_size;
        
        self.ident = torch.eye(vocab_size)
        self.rnn = nn.LSTM(vocab_size, hidden_size, hidden_layers, batch_first=True)
        
    def forward(self, target, hidden=None):
        target_tensor = self.ident[target] # Type: batch.context[0] | Size: (batch size, sequence size)
        out, last_hidden = self.rnn(target_tensor)
        
        return out, last_hidden;
    
class DecoderAE(nn.Module):
    def __init__(self,
                vocab_size,
                hidden_size = 100,
                hidden_layers = 1):
        
        super(DecoderAE, self).__init__()
        
        self.vocab_size = vocab_size;
        self.layers = hidden_layers;
        self.hidden_size = hidden_size;
        
        self.ident = torch.eye(vocab_size)
        self.rnn = nn.LSTM(vocab_size, hidden_size, hidden_layers, batch_first=True)
        self.fcnn = nn.Linear(hidden_size, vocab_size)
        
    def forward(self, target, embedding, hidden=None):
        target_tensor =  self.ident[target]
        if(hidden == None):
            out, last_hidden = self.rnn(target_tensor, embedding)
        else:
            out, last_hidden = self.rnn(target_tensor, hidden)
        out_final = self.fcnn(out)
        return out_final, last_hidden
    
class ContextAE(nn.Module):
    def __init__(self, 
                 context_vocab_size, 
                 encoder_hidden_size = 100,
                 generator_hidden_size = 100, 
                 encoder_layers = 1, 
                 generator_layers = 1):
        
        super(ContextAE, self).__init__()
        
        self.encoder = EncoderAE(context_vocab_size,encoder_hidden_size,encoder_layers)
        self.decoder = DecoderAE(context_vocab_size,generator_hidden_size,generator_layers)
        
    def forward(self, target, hidden = None):
        encode_out, encode_last_hidden = self.encoder(target, hidden)
        decode_out, decode_last_hidden = self.decoder(target, encode_last_hidden,hidden)
        
        return decode_out, decode_last_hidden

In [0]:
ContextAEModel = ContextAE(input_vocab_size, encoder_hidden_size = 100, generator_hidden_size =100, encoder_layers = 2, generator_layers = 2)
ContextAEModel.load_state_dict(torch.load('/content/gdrive/My Drive/Chatbot/CONTEXT_AE_WEIGHTS_12'))
ContextAEModel = ContextAEModel.cuda();
ContextAEModel.encoder.ident = ContextAEModel.encoder.ident.cuda();
ContextAEModel.decoder.ident = ContextAEModel.decoder.ident.cuda();

In [0]:
ContextAEModel

ContextAE(
  (encoder): EncoderAE(
    (rnn): LSTM(95, 100, num_layers=2, batch_first=True)
  )
  (decoder): DecoderAE(
    (rnn): LSTM(95, 100, num_layers=2, batch_first=True)
    (fcnn): Linear(in_features=100, out_features=95, bias=True)
  )
)

In [0]:
###WARNING : THIS IS THE SLOWEST FUNCTION
def baseLineModel(model, inp):
    

    #Get input embedding    
    inp = ["<BOS>"] + list(inp) + ["<EOS>"]; # inp is a string
    inp_indices = [text_field.vocab.stoi[ch] for ch in inp];
    inp_tensor = torch.Tensor(inp_indices).long().unsqueeze(0);
    _, inp_embeddings = model(inp_tensor)
    inp_embeddings = inp_embeddings[0].squeeze(0)
    inp_embeddings = torch.reshape(inp_embeddings, (-1, 200))
    
    
    
    max_similarity = 0;
    reply = "";
    
    i = 0;
    
    for data in dataset:
        #print("here 1")
        data_inp = ["<BOS>"] + list(data.context) + ["<EOS>"]; # inp is a string
        data_inp_indices = [text_field.vocab.stoi[ch] for ch in data_inp];
        data_inp_tensor = torch.Tensor(data_inp_indices).long().unsqueeze(0);
        _, emb = model(data_inp_tensor);
        embed = emb[0].squeeze(0)
        embed = torch.reshape(embed, (-1, 200))
        #print("here 2")
        cosine_sim = torch.cosine_similarity(inp_embeddings, embed);
        #print("here 3")
        #print(cosine_sim)
        #print(cosine_sim , "Context : ", context)
        if(cosine_sim > max_similarity):
            max_similarity = cosine_sim;
            reply = data.reply;
            #print(reply);

    return reply;

In [0]:
baseLineModel(ContextAEModel.encoder, "Who makes the call? Congress or the fcc?")

In [0]:
def baseline(input):
    response = baseLineModel(ContextAEModel.encoder, "Who makes the call? Congress or the fcc?")
    return response;

# >>> Validation

In [0]:
ResponseAEModel = ContextAE(input_vocab_size, encoder_hidden_size = 100, generator_hidden_size =100, encoder_layers = 1, generator_layers = 1)
ResponseAEModel.load_state_dict(torch.load('/content/gdrive/My Drive/Chatbot/RESPONSE_AE_WEIGHTS_17'))
ResponseAEModel = ResponseAEModel.cuda();
ResponseAEModel.encoder.ident = ResponseAEModel.encoder.ident.cuda();
ResponseAEModel.decoder.ident = ResponseAEModel.decoder.ident.cuda();

In [0]:
val_data = torchtext.data.TabularDataset("/content/gdrive/My Drive/Chatbot/validation.txt", "tsv", fields);

In [0]:
total = 0;
seq2seq_score = 0;
baseline_score = 0;

for data in val_data:
    total += 1;
    seq2seq_response = seq2seq(data.context)
    baseline_response = baseline(data.context);
    actual_response = data.reply;
    
    seq2seq_response_onehot = torch.Tensor([label_field.vocab.stoi[ch] for ch in (["<BOS>"] + list(seq2seq_response) + ["<EOS>"])]).long().unsqueeze(0);
    baseline_response_onehot = torch.Tensor([label_field.vocab.stoi[ch] for ch in (["<BOS>"] + list(baseline_response) + ["<EOS>"])]).long().unsqueeze(0);
    actual_response_onehot = torch.Tensor([label_field.vocab.stoi[ch] for ch in (["<BOS>"] + list(actual_response) + ["<EOS>"])]).long().unsqueeze(0);
    
    _, seq2seq_response_emb = ResponseAEModel(seq2seq_response_onehot)
    _, baseline_response_emb = ResponseAEModel(baseline_response_onehot)
    _, actual_response_emb =ResponseAEModel(actual_response_onehot)
    
    if (torch.cosine_similarity(seq2seq_response_emb, actual_response_emb) > torch.cosine_similarity(baseline_response_emb, actual_response_emb)):
        seq2seq_score += 1;
    else:
        baseline_score += 1;
    

In [0]:
print("Seq2Seq % : ", (seq2seq_score/total) * 100);
print("Baseline % : ", (baseline_score/total) * 100);